In [4]:
import pandas as pd
import numpy as np

# Load the red wine dataset
df = pd.read_csv('../dataset/winequality-red.csv', sep=';')

# Exploratory Data Analysis (EDA) for Red Wine Dataset

In [ ]:
# Get descriptive statistics for all columns
desc = df.describe(percentiles=[0.25, 0.5, 0.75]).T
desc['total_data'] = len(df)

# Reorder columns for clarity
desc = desc[['total_data', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

display(desc)

,total_data,mean,std,min,25%,50%,75%,max
fixed acidity,1599,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [29]:
# Finding outliers
print('Outlier summary for each faetures:')
for col in df.columns:
    if df[col].dtype != 'object':
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        print(f"- {col}: {outliers.shape[0]} outliers (bounds: {lower_bound:.2f}, {upper_bound:.2f})")

Outlier summary for each faetures:
- fixed acidity: 49 outliers (bounds: 3.95, 12.35)
- volatile acidity: 19 outliers (bounds: 0.02, 1.02)
- citric acid: 1 outliers (bounds: -0.40, 0.91)
- residual sugar: 155 outliers (bounds: 0.85, 3.65)
- chlorides: 112 outliers (bounds: 0.04, 0.12)
- free sulfur dioxide: 30 outliers (bounds: -14.00, 42.00)
- total sulfur dioxide: 55 outliers (bounds: -38.00, 122.00)
- density: 45 outliers (bounds: 0.99, 1.00)
- pH: 35 outliers (bounds: 2.92, 3.68)
- sulphates: 59 outliers (bounds: 0.28, 1.00)
- alcohol: 13 outliers (bounds: 7.10, 13.50)
- quality: 28 outliers (bounds: 3.50, 7.50)


# PreProcessing Data

In [5]:
### Data Cleaning

# Missing values handler
missing = df.isnull().sum()
print('Missing values per column:')
display(missing)

# Duplicate rows handler
duplicates = df.duplicated().sum()
print(f'Number of duplicate rows: {duplicates}')
df_cleaned = df.drop_duplicates()

Missing values per column:


fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Number of duplicate rows: 240


In [6]:
### Data Standardization
numeric_cols = [col for col in df_cleaned.select_dtypes(include=np.number).columns if col != 'quality']
df_standardized = df_cleaned.copy()
df_standardized[numeric_cols] = (df_cleaned[numeric_cols] - df_cleaned[numeric_cols].mean()) / df_cleaned[numeric_cols].std()
print("Data successfully standardized.")

Data successfully standardized.


In [7]:
### Correlation matrix
correlation_matrix = df_standardized.corr()
display(correlation_matrix)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
fixed acidity,1.000000,-0.255124,0.667437,0.111025,0.085886,-0.140580,-0.103777,0.670195,-0.686685,0.190269,-0.061596,0.119024
volatile acidity,-0.255124,1.000000,-0.551248,-0.002449,0.055154,-0.020945,0.071701,0.023943,0.247111,-0.256948,-0.197812,-0.395214
citric acid,0.667437,-0.551248,1.000000,0.143892,0.210195,-0.048004,0.047358,0.357962,-0.550310,0.326062,0.105108,0.228057
residual sugar,0.111025,-0.002449,0.143892,1.000000,0.026656,0.160527,0.201038,0.324522,-0.083143,-0.011837,0.063281,0.013640
chlorides,0.085886,0.055154,0.210195,0.026656,1.000000,0.000749,0.045773,0.193592,-0.270893,0.394557,-0.223824,-0.130988
free sulfur dioxide,-0.140580,-0.020945,-0.048004,0.160527,0.000749,1.000000,0.667246,-0.018071,0.056631,0.054126,-0.080125,-0.050463
total sulfur dioxide,-0.103777,0.071701,0.047358,0.201038,0.045773,0.667246,1.000000,0.078141,-0.079257,0.035291,-0.217829,-0.177855
density,0.670195,0.023943,0.357962,0.324522,0.193592,-0.018071,0.078141,1.000000,-0.355617,0.146036,-0.504995,-0.184252
pH,-0.686685,0.247111,-0.550310,-0.083143,-0.270893,0.056631,-0.079257,-0.355617,1.000000,-0.214134,0.213418,-0.055245
sulphates,0.190269,-0.256948,0.326062,-0.011837,0.394557,0.054126,0.035291,0.146036,-0.214134,1.000000,0.091621,0.248835


In [8]:
# Feature Selection
df_selected = df_standardized[['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'total sulfur dioxide', 'sulphates', 'alcohol', 'quality']]
display(df_selected)

,fixed acidity,volatile acidity,residual sugar,chlorides,total sulfur dioxide,sulphates,alcohol,quality
0,-0.524238,0.931657,-0.460987,-0.245532,-0.383908,-0.578348,-0.954023,5
1,-0.293955,1.915095,0.056644,0.200020,0.603851,0.124776,-0.584360,5
2,-0.293955,1.259470,-0.165198,0.078506,0.214734,-0.051005,-0.584360,5
3,1.663455,-1.363032,-0.460987,-0.265785,0.394326,-0.461161,-0.584360,6
5,-0.524238,0.713115,-0.534935,-0.265785,-0.204316,-0.578348,-0.954023,5
...,...,...,...,...,...,...,...,...
1593,-0.869663,0.494574,-0.460987,-0.407552,-0.264180,0.945087,-0.861607,6
1594,-1.215088,0.385303,-0.387040,0.038001,-0.084587,-0.461161,0.062551,5
1595,-1.387801,0.112125,-0.239145,-0.529066,0.124937,0.593525,0.709462,6
1597,-1.387801,0.631162,-0.387040,-0.265785,-0.084587,0.300557,-0.214696,5


In [9]:
# Check for Balance
quality_counts = df_selected['quality'].value_counts().sort_index()
print("Quality value counts:")
display(quality_counts)

Quality value counts:


quality
3     10
4     53
5    577
6    535
7    167
8     17
Name: count, dtype: int64

In [ ]:
# Quality classification (0 = low, 1 = high)
df_classified = df_selected.copy()
df_classified['quality'] = (df_classified['quality'] >= 6).astype(int)
quality_counts = df_classified['quality'].value_counts().sort_index()
print("Quality value counts:")
display(quality_counts)

Quality value counts:


quality
0    1175
1     184
Name: count, dtype: int64

In [11]:
# Save processed data
df_classified.to_csv('../dataset/winequality-red-preprocessed.csv', index=False)